In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
from shapely.geometry import Polygon
os.environ['USE_PYGEOS'] = '0'
import geopandas
import shutil
import numpy as np
import napari
import os
import spatialdata as sd
from spatialdata._core.core_utils import get_default_coordinate_system
from spatialdata._core.transformations import Affine, Sequence, Identity
from spatialdata._core._transform_elements import align_elements_using_landmarks
from spatialdata._core._spatialdata_ops import get_transformation, set_transformation
from napari_spatialdata import Interactive

/Users/b450-admin/opt/miniconda3/envs/napari-env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
DATA_PATH = "data"

XE_REP1_ROI_ST_PATH     = os.path.join(DATA_PATH, "xe_rep1_roi_st.zarr")
XE_REP2_ROI_ST_PATH     = os.path.join(DATA_PATH, "xe_rep2_roi_st.zarr")
VISIUM_ROI_ST_PATH      = os.path.join(DATA_PATH, "visium_roi_st.zarr")
# VISIUM_PATH      = os.path.join(DATA_PATH, "visium.zarr")

assert os.path.isdir(XE_REP1_ROI_ST_PATH)
assert os.path.isdir(XE_REP2_ROI_ST_PATH)
assert os.path.isdir(VISIUM_ROI_ST_PATH)

In [12]:
xe_rep1_roi_st_sdata     = sd.read_zarr(XE_REP1_ROI_ST_PATH)
xe_rep2_roi_st_sdata     = sd.read_zarr(XE_REP2_ROI_ST_PATH)
visium_roi_st_sdata      = sd.read_zarr(VISIUM_ROI_ST_PATH)

/Users/b450-admin/opt/miniconda3/envs/napari-env/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [58]:
from spatialdata import ShapesModel, TableModel
xe_rep1_roi_st_sdata.shapes['ROIs']['ROI'] = ['ROI_' + str(x+1) for x in xe_rep1_roi_st_sdata.shapes['ROIs'].index]
xe_rep1_roi_st_sdata.shapes['cell_boundaries']['cell_id'] = xe_rep1_roi_st_sdata.table.obs['cell_id'].values

rois_xe_rep1 = xe_rep1_roi_st_sdata.shapes['ROIs'].overlay(xe_rep1_roi_st_sdata.shapes['cell_boundaries'], how='intersection')
rois_xe_rep1.drop_duplicates(subset=['cell_id'], keep='first')
rois_xe_rep1 = rois_xe_rep1.drop_duplicates(subset=['cell_id'], keep='first')

xe_rep1_roi_table = xe_rep1_roi_st_sdata.table[xe_rep1_table['cell_id'].astype(str),:]
xe_rep1_roi_table.obs['ROI'] = rois_xe_rep1['ROI'].values

rois_xe_rep1 = ShapesModel.parse(rois_xe_rep1, transformations={"global": get_transformation(xe_rep1_roi_st_sdata.shapes['ROIs'], 'global')})
xe_rep1_roi_table = TableModel.parse(xe_rep1_roi_table)

xe_rep1_subset_sdata = sd.SpatialData(
    shapes={
        "ROIs": rois_xe_rep1
    },
    table=xe_rep1_roi_table
    
)


xe_rep1_subset_sdata.write('data/xe_rep1_subset_roi.zarr')

/var/folders/5x/ypt5v1fx6rs06gz37q_85t900000gp/T/ipykernel_73129/3394046582.py:10: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  xe_rep1_roi_table.obs['ROI'] = rois_xe_rep1['ROI'].values
/Users/b450-admin/opt/miniconda3/envs/napari-env/lib/python3.9/site-packages/zarr/creation.py:250: UserWarning: ignoring keyword argument 'maxshape'
  warn('ignoring keyword argument %r' % k)


In [61]:
xe_rep2_roi_st_sdata.shapes['ROIs']['ROI'] = ['ROI_' + str(x+1) for x in xe_rep2_roi_st_sdata.shapes['ROIs'].index]
xe_rep2_roi_st_sdata.shapes['cell_boundaries']['cell_id'] = xe_rep2_roi_st_sdata.table.obs['cell_id'].values
rois_xe_rep2 = xe_rep2_roi_st_sdata.shapes['ROIs'].overlay(xe_rep2_roi_st_sdata.shapes['cell_boundaries'], how='intersection')
xe_rep2_roi_st_sdata.table.obs['ROI'] = 'NA'
rois_xe_rep2 = rois_xe_rep2.drop_duplicates(subset=['cell_id'], keep='first')
xe_rep2_roi_table = xe_rep2_roi_st_sdata.table[xe_rep2_table['cell_id'].astype(str),:]
xe_rep2_roi_table.obs['ROI'] = rois_xe_rep2['ROI'].values

rois_xe_rep2 = ShapesModel.parse(rois_xe_rep2, transformations={"global": get_transformation(xe_rep2_roi_st_sdata.shapes['ROIs'], 'global')})
xe_rep2_roi_table = TableModel.parse(xe_rep2_roi_table)

xe_rep2_subset_sdata = sd.SpatialData(
    shapes={
        "ROIs": rois_xe_rep2
    },
    table=xe_rep2_roi_table
    
)


xe_rep2_subset_sdata.write('data/xe_rep2_subset_roi.zarr')

/var/folders/5x/ypt5v1fx6rs06gz37q_85t900000gp/T/ipykernel_73129/1101808495.py:7: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  xe_rep2_roi_table.obs['ROI'] = rois_xe_rep2['ROI'].values
/Users/b450-admin/opt/miniconda3/envs/napari-env/lib/python3.9/site-packages/zarr/creation.py:250: UserWarning: ignoring keyword argument 'maxshape'
  warn('ignoring keyword argument %r' % k)


In [63]:
visium_roi_st_sdata.shapes['ROIs']['ROI'] = ['ROI_' + str(x+1) for x in visium_roi_st_sdata.shapes['ROIs'].index]
visium_roi_st_sdata.shapes['visium']['spot_id'] = visium_roi_st_sdata.table.obs['spot_id'].values
rois_visium = visium_roi_st_sdata.shapes['ROIs'].overlay(visium_roi_st_sdata.shapes['visium'], how='intersection')
visium_roi_st_sdata.table.obs['ROI'] = 'NA'
rois_visium = rois_visium.drop_duplicates(subset=['spot_id'], keep='first')
visium_roi_table = visium_roi_st_sdata.table[visium_table['spot_id'].astype(str),:]
visium_roi_table.obs['ROI'] = rois_visium['ROI'].values

rois_visium = ShapesModel.parse(rois_visium, transformations={"global": get_transformation(visium_roi_st_sdata.shapes['ROIs'], 'global')})
visium_roi_table = TableModel.parse(visium_roi_table)

visium_subset_sdata = sd.SpatialData(
    shapes={
        "ROIs": rois_visium
    },
    table=visium_roi_table
    
)



visium_subset_sdata.write('data/visium_subset_roi.zarr')

/var/folders/5x/ypt5v1fx6rs06gz37q_85t900000gp/T/ipykernel_73129/1592965037.py:7: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  visium_roi_table.obs['ROI'] = rois_visium['ROI'].values
/Users/b450-admin/opt/miniconda3/envs/napari-env/lib/python3.9/site-packages/zarr/creation.py:250: UserWarning: ignoring keyword argument 'maxshape'
  warn('ignoring keyword argument %r' % k)
